In [1]:
#to get all data from drive into collab for current session
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
#complete code - OpenCV Plan A
#Accuracy - WT = 161/170 = 94.7%;  MUT = 59/60 = 98.33%;   IND = 100%;
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from glob import glob
u=0
ind=0
mut=0
wt=0
mt=0
input_images = glob('/content/drive/MyDrive/Wild-type/*.tif') #change location for various dataset
for j in input_images:
  img = cv2.imread(j)
  roi = img[200:475,125:200,:] #standard for all - will cause minute error in some
  ig = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
  ib = cv2.adaptiveThreshold(ig,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,89,4)
  ed = cv2.Canny(ib,90,50) #edge detection
  c=0
  l=0
  f=0
  len=[] #length between 2 white pixels (of rectangle)
  s=roi.shape
  i = ed[:,s[1]//2:(s[1]//2)+1] #taking mid column
  for x in range(s[0]-2):
    l=l+1
    if i[x][0] == 255 and i[x+1][0] != 255:#non-consequtive pixels
      c=c+1 #counting no. of white pixels
      len.append(l)#calculating lenght b/w edges
      l=0
  if c<=3:
    ind=ind+1
  elif c>=6:
    mut=mut+1 #for MUT 1
  else:
    for k in len: #for differenciating b/w WT & MUT 2
      if k>(s[0]//2):
        f=f+1
    if f==0:
      wt=wt+1
    else:
      mut=mut+1 #for MUT 2
print("IND: " ,ind, "; MUT: ",mut,"; WT: ",wt)

In [ ]:
#complete code - using Nuttada's RGB to Gray Vector
#Accuracy - WT = 162/170 = 95.29%;   MUT = 58/60 = 96.66%;   IND =  100%;
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from glob import glob
u=0
ind=0
mut=0
wt=0
mt=0
input_images = glob('/content/drive/MyDrive/Mutant/*.tif')
for j in input_images:
  img = cv2.imread(j)
  roi = img[200:475,125:200,:] #standard for all
  ch = (3.4663*roi[:,:,0] + 2.7478*roi[:,:,1] + 0.8025*roi[:,:,2])/7.0166 #converting to grayscale using Nuttada's LDA vector
  ch = np.reshape(ch,[275,75,1])#reshaping to get 1 channel
  cv2.imwrite('single_channel.jpg', ch)
  ig = cv2.imread("single_channel.jpg",0)#to convert to cv2 object so that it can be used as input in thresholding - grayscale image
  ib = cv2.adaptiveThreshold(ig,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,89,4)
  ed = cv2.Canny(ib,90,50) #edge detection
  s=roi.shape
  i = ed[:,s[1]//2:(s[1]//2)+1] #taking mid column
  c=0
  l=0
  f=0
  len=[] #length between 2 white pixels (of rectangle)
  for x in range(s[0]-2):
    l=l+1
    if i[x][0] == 255 and i[x+1][0] != 255:
      c=c+1 #counting no. of white
      len.append(l)
      l=0
  if c<=3:
    ind=ind+1
  elif c>=6:
    mut=mut+1
  else:
    for k in len:
      if k>(s[0]//2):
        f=f+1
    if f==0:
      wt=wt+1
    else:
      mut=mut+1
print("IND: " ,ind, "; MUT: ",mut,"; WT: ",wt)

In [ ]:
#optimising number of columns
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from glob import glob
from statistics import mean
from PIL import Image, ImageEnhance
input_images = glob('/content/drive/MyDrive/Wild-type/*.tif') #change location for various dataset
accuracy=[]
col=3
noc=[]
for n in range(25,37):#to run from 1/3rd to 1/2
  u=0
  ind=0
  mut=0
  wt=0
  for j in input_images:
    img = cv2.imread(j)
    roi = img[200:475,125:200,:] #standard for all - will cause minute error in some
    ig = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    ib = cv2.adaptiveThreshold(ig,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,89,4)
    ed = cv2.Canny(ib,90,50) #edge detection
    s=roi.shape
    i = ed[:,s[1]//2:(s[1]//2)+1] #taking mid column
    c=0
    edges=[]
    l=0
    f=0
    len=[] #length between 2 white pixels (of rectangle)
    #for middle column extraction and counting
    for x in range(s[0]-2):
      l=l+1
      if i[x][0] == 255 and i[x+1][0] != 255:
        c=c+1 #counting no. of white
        len.append(l)
        l=0
    edges.append(c)
    lenght=[]
    lenght.append(max(len))
    #to extract nth column
    t=((2*s[1])//3)+1
    for x in range(s[1]//3,n+1):
      i1=ed[:,x:x+1]
      i2 = ed[:,t-1:t]
      len1=[]
      len2=[]
      l1=0
      l2=0
      c1=0
      c2=0
      for x in range(s[0]-2):
        l1=l1+1
        l2=l2+1
        if i1[x][0] == 255 and i1[x+1][0] != 255:
          c1=c1+1 #counting no. of white
          len1.append(l1)
          l1=0
        if i2[x][0] == 255 and i2[x+1][0] != 255:
          c2=c2+1 #counting no. of white
          len2.append(l2)
          l2=0
      lenght.append(max(len1))
      lenght.append(max(len2))
      edges.append(c1)
      edges.append(c2)
      t=t-1
    count = round(mean(edges))
    if count<=3:
      ind=ind+1
    elif count>=6:
      mut=mut+1
    else:
      for k in lenght:
        if k>(s[0]//2):
          f=f+1
      if f==0:
        wt=wt+1
      else:
        mut=mut+1
  noc.append(col)
  col=col+2
  accuracy.append((wt/170)*100)
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
plt.title('Wild-type',fontweight='bold')
plt.xlabel('Number of columns',fontweight='bold')
plt.ylabel('Accuracy',fontweight='bold')
plt.ylim(95, 100)
plt.plot(noc, accuracy,'-o')
#plt.savefig('destination_path.eps', format='eps')#for eps format instead on png
plt.savefig('WT.png')